In [15]:
import pandas as pd

1. Look at the PBA50+ BAUS zoning for parcels

In [16]:
pz = pd.read_csv("M:/urban_modeling/baus/PBA50Plus/Draft_Blueprint/\
PBA50Plus_Draft_Blueprint_v8_znupd_nodevfix_interimoutput/core_summaries/\
PBA50Plus_Draft_Blueprint_v8_znupd_nodevfix_interimoutput_parcel_summary_2050.csv")

2. Add growth geography variables for understanding parcels' location

In [17]:
# parcels geography input file
pg = pd.read_csv("M:/urban_modeling/baus/BAUS Inputs/basis_inputs/crosswalks/parcels_geography_2024_02_14.csv")

pz = pz.merge(pg[['PARCEL_ID', 'juris', 'gg_id', 'pda_id', 'tra_id', 'hra_id']], 
                  left_on='parcel_id', right_on='PARCEL_ID', how='left')

3. Add the max built DUA/FAR in a parcel's TAZ, used in the BAUS zoning processing

In [18]:
# parcel to TAZ crosswalk
ptaz = pd.read_csv("M:/urban_modeling/baus/BAUS Inputs/basis_inputs/crosswalks/2020_08_17_parcel_to_taz1454sub.csv")
pz = pz.merge(ptaz[['PARCEL_ID', 'ZONE_ID']], left_on='parcel_id', right_on='PARCEL_ID', how='left')

max_dua_builtdua_taz = pz.built_dua.groupby(pz.ZONE_ID).max().fillna(4)
pz['max_dua_builtdua_taz'] = pz['ZONE_ID'].map(max_dua_builtdua_taz).fillna(4)

max_far_builtdua_taz = pz.built_far.groupby(pz.ZONE_ID).max().fillna(4)
pz['max_dua_builtfar_taz'] = pz['ZONE_ID'].map(max_far_builtdua_taz).fillna(4)

4. Add the base zoning to parcels in order to lookup the corresponding zoning input

In [19]:
# latest parcels zoning file
pzon = pd.read_csv("M:/urban_modeling/baus/BAUS Inputs/basis_inputs/zoning/zoning_parcels_2024-04-30.csv")
# latest parcel lookup file
zl = pd.read_csv("M:/urban_modeling/baus/BAUS Inputs/basis_inputs/zoning/zoning_lookup_2024-04-30.csv")
zl = zl.rename(columns={"name": "zoning_name", "max_dua": "max_dua_zoning", "max_far": "max_far_zoning"})

# merge the zoning lookup to parcels
pz = pz.merge(pzon[['PARCEL_ID', 'zoning_id']], left_on='parcel_id', right_on='PARCEL_ID', how='left')\
.merge(zl[['id', 'zoning_name', 'max_dua_zoning', 'max_far_zoning']], left_on='zoning_id', right_on='id', how='left')

5. Inspect the BAUS zoning based on the processing steps

BAUS DUA processing steps / zoning variables in the table:
- **max_dua_zoning** is the input zoning
- **max_dua_imputation** is zoning with imputation and residential allowance applied
- **max_dua_imputation_upzoning** is zoning with imputation and upzoning and residential allowance applied
- **maz_dua_builtfar_taz** is the highest max DUA in the TAZ
- **max_dua** is zoning with imputation and upzoning, residenital allowance, nodev modification, outside of urban area modification, \
and lowest built density modification applied

### DUA

In [20]:
print("Number of parcels with DUA > 40 is {}".format(len(pz.loc[pz.max_dua > 40])))
dua = pz.loc[pz.max_dua > 40][['parcel_id', 'zoning_name', 'max_dua_zoning', 'juris', 'tra_id', 'pda_id',
                               'max_dua_imputation', 'max_dua_imputation_upzoning', 'max_dua_builtdua_taz', 'max_dua', 'built_dua']].\
                                sort_values(by='max_dua', ascending=False)

Number of parcels with DUA > 40 is 248983


#### DUA from Upzoning

In [21]:
dua_upzoning = dua.loc[(dua.max_dua_imputation_upzoning > dua.max_dua_imputation) & (dua.max_dua_imputation_upzoning == dua.max_dua)]
print("Number of parcels with DUA > 200 from upzoning step is {}".format(len(dua_upzoning.loc[dua_upzoning.max_dua > 200])))
dua_upzoning.loc[dua_upzoning.max_dua > 200].head()

Number of parcels with DUA > 200 from upzoning step is 0


,parcel_id,zoning_name,max_dua_zoning,juris,tra_id,pda_id,max_dua_imputation,max_dua_imputation_upzoning,max_dua_builtdua_taz,max_dua,built_dua


In [22]:
dua_upzoning_high_nongg = dua_upzoning.loc[(dua_upzoning.max_dua > 200) & (dua_upzoning.tra_id.isnull()) & (dua_upzoning.pda_id.isnull())]
print("Number of parcels with DUA > 200 from upzoning step but not in a TRA or PDA is {}".format(len(dua_upzoning_high_nongg)))
dua_upzoning_high_nongg.head()

Number of parcels with DUA > 200 from upzoning step but not in a TRA or PDA is 0


,parcel_id,zoning_name,max_dua_zoning,juris,tra_id,pda_id,max_dua_imputation,max_dua_imputation_upzoning,max_dua_builtdua_taz,max_dua,built_dua


#### DUA from Imputation

In [23]:
dua_imputation = dua.loc[(dua.max_dua_imputation > dua.max_dua_zoning) & (dua.max_dua_imputation == dua.max_dua)]
print("Number of parcels with DUA > 40 from imputation is {}".format(len(dua_imputation.loc[dua_imputation.max_dua > 40])))
dua_imputation.loc[dua_imputation.max_dua > 40].head()

Number of parcels with DUA > 40 from imputation is 0


,parcel_id,zoning_name,max_dua_zoning,juris,tra_id,pda_id,max_dua_imputation,max_dua_imputation_upzoning,max_dua_builtdua_taz,max_dua,built_dua


In [24]:
dua_imputation_high_nongg = dua_imputation.loc[(dua_imputation.max_dua > 40) & (dua_imputation.tra_id.isnull()) & (dua_imputation.pda_id.isnull())]
print("Number of parcels with DUA > 40 from imputation and not in a TRA or PDA is {}".format(len(dua_imputation_high_nongg)))
dua_imputation_high_nongg

Number of parcels with DUA > 40 from imputation and not in a TRA or PDA is 0


,parcel_id,zoning_name,max_dua_zoning,juris,tra_id,pda_id,max_dua_imputation,max_dua_imputation_upzoning,max_dua_builtdua_taz,max_dua,built_dua


#### DUA from Max TAZ Built DUA

In [25]:
dua_builtdua_taz_max = dua.loc[(dua.max_dua_builtdua_taz == dua.max_dua) & (dua.max_dua_builtdua_taz > dua.max_dua_zoning)]
print("Number of parcels with DUA > 40 set by building to the lowest built DUA {}".format(len(dua_builtdua_taz_max.loc[dua_builtdua_taz_max.max_dua > 40])))
dua_builtdua_taz_max.loc[dua_builtdua_taz_max.max_dua > 40].head()

Number of parcels with DUA > 40 set by building to the lowest built DUA 5021


,parcel_id,zoning_name,max_dua_zoning,juris,tra_id,pda_id,max_dua_imputation,max_dua_imputation_upzoning,max_dua_builtdua_taz,max_dua,built_dua
1758718,1105756,RRMU (BASIS),70.0,burlingame,TRA1,Burlingame - Downtown,43.56,200.0,191.192672,191.192672,0.0
359713,1105761,RRMU (BASIS),70.0,burlingame,TRA1,Burlingame - Downtown,43.56,200.0,191.192672,191.192672,0.0
606446,1105777,RRMU (BASIS),70.0,burlingame,TRA1,Burlingame - Downtown,43.56,200.0,191.192672,191.192672,0.0
267662,1105787,RRMU (BASIS),70.0,burlingame,TRA1,Burlingame - Downtown,43.56,200.0,191.192672,191.192672,0.0
159403,1105759,RRMU (BASIS),70.0,burlingame,TRA1,Burlingame - Burlingame El Camino Real,43.56,200.0,191.192672,191.192672,0.0


In [26]:
dua_builtdua_taz_max_high_nongg = dua_builtdua_taz_max.loc[(dua_builtdua_taz_max.max_dua > 40) & 
                                  (dua_builtdua_taz_max.tra_id.isnull()) & (dua_builtdua_taz_max.pda_id.isnull())]
print("Number of parcels with DUA > 40 from max built dua in TAZ and not in a TRA or PDA is {}".format(len(dua_builtdua_taz_max_high_nongg)))
dua_builtdua_taz_max_high_nongg.head()

Number of parcels with DUA > 40 from max built dua in TAZ and not in a TRA or PDA is 0


,parcel_id,zoning_name,max_dua_zoning,juris,tra_id,pda_id,max_dua_imputation,max_dua_imputation_upzoning,max_dua_builtdua_taz,max_dua,built_dua


#### DUA from Zoning Input

In [27]:
dua_zoning = dua.loc[(dua.max_dua_zoning == dua.max_dua)]
print("Number of parcels with DUA > 40 from input zoning is {}".format(len(dua_zoning)))
dua_zoning.head()

Number of parcels with DUA > 40 from input zoning is 72205


,parcel_id,zoning_name,max_dua_zoning,juris,tra_id,pda_id,max_dua_imputation,max_dua_imputation_upzoning,max_dua_builtdua_taz,max_dua,built_dua
1597068,981784,RH DTR (HE Data),1559.7,san_francisco,TRA1,San Francisco - Transbay/Rincon Hill,1559.7,1559.7,2409.727461,1559.7,0.000000
169064,981785,RH DTR (HE Data),1559.7,san_francisco,TRA1,San Francisco - Transbay/Rincon Hill,1559.7,1559.7,2409.727461,1559.7,0.000000
502998,981681,TB DTR (HE Data),1559.7,san_francisco,TRA1,San Francisco - Transbay/Rincon Hill,1559.7,1559.7,2409.727461,1559.7,0.000000
85050,981723,TB DTR (HE Data),1538.1,san_francisco,TRA1,San Francisco - Transbay/Rincon Hill,1538.1,1538.1,2409.727461,1538.1,79.177298
1669360,982865,CMUO (HE Data),1536.8,san_francisco,TRA2,San Francisco - Eastern Neighborhoods,1536.8,1536.8,2052.537124,1536.8,22.909390


In [28]:
dua_zoning_high_nongg = dua_zoning.loc[(dua_zoning.max_dua > 40) & 
                                       ((dua_zoning.tra_id.isnull()) & (dua_zoning.pda_id.isnull()))]
print("Number of parcels with DUA > 40 from input zoning and not in a TRA or PDA is {}".format(len(dua_zoning_high_nongg)))
dua_zoning_high_nongg.head()

Number of parcels with DUA > 40 from input zoning and not in a TRA or PDA is 16721


,parcel_id,zoning_name,max_dua_zoning,juris,tra_id,pda_id,max_dua_imputation,max_dua_imputation_upzoning,max_dua_builtdua_taz,max_dua,built_dua
896229,900474,85' Height Allowed (HE Data),243.9,san_francisco,NaN,NaN,243.9,243.9,345.596402,243.9,0.000000
209288,900576,85' Height Allowed (HE Data),243.9,san_francisco,NaN,NaN,243.9,243.9,292.793462,243.9,0.000000
1178237,907017,85' Height Allowed (HE Data),243.9,san_francisco,NaN,NaN,243.9,243.9,292.793462,243.9,0.000000
1052216,900580,85' Height Allowed (HE Data),243.9,san_francisco,NaN,NaN,243.9,243.9,292.793462,243.9,18.363485
1662747,900478,85' Height Allowed (HE Data),243.9,san_francisco,NaN,NaN,243.9,243.9,345.596402,243.9,0.000000


In [29]:
dua_zoning_high_nongg_nonbig3 = dua_zoning.loc[(dua_zoning.parcel_id.isin(dua_zoning_high_nongg.parcel_id)) & 
            (dua_zoning.juris != 'san_francisco') & (dua_zoning.juris != 'san_jose') & (dua_zoning.juris != 'oakland')]
print("Number of parcels with DUA > 40 from zoning input and not in a TRA, PDA, or Big 3 is {}".format(len(dua_zoning_high_nongg_nonbig3)))
dua_zoning_high_nongg_nonbig3.head()

Number of parcels with DUA > 40 from zoning input and not in a TRA, PDA, or Big 3 is 3411


,parcel_id,zoning_name,max_dua_zoning,juris,tra_id,pda_id,max_dua_imputation,max_dua_imputation_upzoning,max_dua_builtdua_taz,max_dua,built_dua
1492614,1080119,T6UC (HE Data),200.0,south_san_francisco,NaN,NaN,200.0,200.0,951.196835,200.0,0.0
135478,1080046,T5C (HE Data),140.0,south_san_francisco,NaN,NaN,140.0,140.0,951.196835,140.0,0.0
203061,1080158,T5C (HE Data),140.0,south_san_francisco,NaN,NaN,140.0,140.0,951.196835,140.0,0.0
135479,1080121,T5C (HE Data),140.0,south_san_francisco,NaN,NaN,140.0,140.0,951.196835,140.0,0.0
1045726,1080116,T5C (HE Data),140.0,south_san_francisco,NaN,NaN,140.0,140.0,951.196835,140.0,0.0


In [30]:
# export associated zoning codes to check them
#dua_zoning_high_nongg_nonbig3.zoning_name.value_counts().to_csv("high base zoning counts.csv")

In [31]:
# export associated parcels to view location of input zoning on a map (can dissolve on zoning code)
#dua_zoning_high_nongg_nonbig3.to_csv("dua_zoning_high_nongg_nonbig3.csv")

### FAR


BAUS FAR processing steps / zoning variables in the table:
- **max_far_zoning** is the input zoning
- **max_far_imputation** is zoning with imputation and residential allowance applied
- **max_far_imputation_upzoning** is zoning with imputation and upzoning and residential allowance applied
- **max_far** is zoning with imputation and upzoning, residenital allowance, nodev modification, outside of urban area modification,\
and lowest built density modification applied

In [41]:
print("Number of parcels with FAR > 4 is {}".format(len(pz.loc[pz.max_far > 4])))
far = pz.loc[pz.max_far > 4][['parcel_id', 'zoning_name', 'max_far_zoning', 'juris', 'tra_id', 'pda_id',
                              'max_far_imputation', 'max_far_imputation_upzoning', 'max_dua_builtfar_taz', 'max_far', 'built_far']].\
                              sort_values(by='max_far', ascending=False).rename(columns={"max_dua_builtfar_taz": "max_far_builtfar_taz"})

Number of parcels with FAR > 4 is 27883


#### FAR from Upzoning

In [43]:
far_upzoning = far.loc[(far.max_far_imputation_upzoning > far.max_far_imputation) & (far.max_far_imputation_upzoning == far.max_far)]
print("Number of parcels with FAR > 4 from upzoning step is {}".format(len(far_upzoning.loc[far_upzoning.max_far > 4])))
far_upzoning.loc[far_upzoning.max_far > 4].head()

Number of parcels with FAR > 4 from upzoning step is 17136


,parcel_id,zoning_name,max_far_zoning,juris,tra_id,pda_id,max_far_imputation,max_far_imputation_upzoning,max_far_builtfar_taz,max_far,built_far
502993,981358,WMUG (HE Data),NaN,san_francisco,TRA1,San Francisco - Eastern Neighborhoods,0.000000,10.0,10.766031,10.0,1.583020
505681,6071,D-LM-2 (BASIS),0.0,oakland,TRA1,Oakland - Downtown & Jack London Square,0.000000,10.0,72.441954,10.0,3.750000
268827,980607,C-3-G (HE Data),NaN,san_francisco,TRA1,San Francisco - Downtown/Van Ness/Northeast Ne...,0.000000,10.0,74.003845,10.0,1.494303
268835,980714,C-3-R (HE Data),NaN,san_francisco,TRA1,San Francisco - Downtown/Van Ness/Northeast Ne...,0.000000,10.0,36.328024,10.0,7.000000
1005884,1103817,MSASP Quasi-Public_pba40,NaN,millbrae,TRA1,Millbrae - Transit Station Area,8.066667,10.0,13.764236,10.0,0.000000


In [44]:
far_upzoning_high_nongg = far_upzoning.loc[(far_upzoning.max_far > 4) & (far_upzoning.tra_id.isnull()) & (far_upzoning.pda_id.isnull())]
print("Number of parcels with FAR > 4 from upzoning step but not in a TRA or PDA is {}".format(len(far_upzoning_high_nongg)))
far_upzoning_high_nongg.head()

Number of parcels with FAR > 4 from upzoning step but not in a TRA or PDA is 0


,parcel_id,zoning_name,max_far_zoning,juris,tra_id,pda_id,max_far_imputation,max_far_imputation_upzoning,max_far_builtfar_taz,max_far,built_far


#### FAR from Imputation

In [45]:
far_imputation = far.loc[(far.max_far_imputation > far.max_far_zoning) & (far.max_far_imputation == far.max_far)]
print("Number of parcels with FAR > 4 from imputation is {}".format(len(far_imputation.loc[far_imputation.max_far > 4])))
far_imputation.loc[far_imputation.max_far > 4].head()

Number of parcels with FAR > 4 from imputation is 0


,parcel_id,zoning_name,max_far_zoning,juris,tra_id,pda_id,max_far_imputation,max_far_imputation_upzoning,max_far_builtfar_taz,max_far,built_far


In [46]:
far_imputation_high_nongg = far_imputation.loc[(far_imputation.max_far > 40) & (far_imputation.tra_id.isnull()) & (far_imputation.pda_id.isnull())]
print("Number of parcels with FAR > 4 from imputation and not in a TRA or PDA is {}".format(len(far_imputation_high_nongg)))
far_imputation_high_nongg

Number of parcels with FAR > 4 from imputation and not in a TRA or PDA is 0


,parcel_id,zoning_name,max_far_zoning,juris,tra_id,pda_id,max_far_imputation,max_far_imputation_upzoning,max_far_builtfar_taz,max_far,built_far


#### FAR from Max TAZ Built FAR

In [47]:
far_builtfar_taz_max = far.loc[(far.max_far_builtfar_taz == far.max_far) & (far.max_far_builtfar_taz > far.max_far_zoning)]
print("Number of parcels with FAR > 4 set by building to the lowest built DUA {}".format(len(far_builtfar_taz_max.loc[far_builtfar_taz_max.max_far > 4])))
far_builtfar_taz_max.loc[far_builtfar_taz_max.max_far > 4].head()

Number of parcels with FAR > 4 set by building to the lowest built DUA 1682


,parcel_id,zoning_name,max_far_zoning,juris,tra_id,pda_id,max_far_imputation,max_far_imputation_upzoning,max_far_builtfar_taz,max_far,built_far
606093,6555,D-BV-2 (BASIS),0.0,oakland,TRA1,Oakland - Downtown & Jack London Square,0.0,10.0,9.942464,9.942464,0.786519
1464981,6564,Urban Residential_pba40,6.0,oakland,TRA1,Oakland - Downtown & Jack London Square,6.0,10.0,9.942464,9.942464,0.000000
489894,6551,D-BV-2 (BASIS),0.0,oakland,TRA1,Oakland - Downtown & Jack London Square,0.0,10.0,9.942464,9.942464,3.750000
1300511,6619,D-BV-2 (BASIS),0.0,oakland,TRA1,Oakland - Downtown & Jack London Square,0.0,10.0,9.942464,9.942464,0.000000
1039160,6544,D-BV-2 (BASIS),0.0,oakland,TRA1,Oakland - Downtown & Jack London Square,0.0,10.0,9.942464,9.942464,8.313090


In [48]:
far_builtfar_taz_max_high_nongg = far_builtfar_taz_max.loc[(far_builtfar_taz_max.max_far > 4) & 
                                  (far_builtfar_taz_max.tra_id.isnull()) & (far_builtfar_taz_max.pda_id.isnull())]
print("Number of parcels with FAR > 4 from max built dua in TAZ and not in a TRA or PDA is {}".format(len(far_builtfar_taz_max_high_nongg)))
far_builtfar_taz_max_high_nongg.head()

Number of parcels with FAR > 4 from max built dua in TAZ and not in a TRA or PDA is 0


,parcel_id,zoning_name,max_far_zoning,juris,tra_id,pda_id,max_far_imputation,max_far_imputation_upzoning,max_far_builtfar_taz,max_far,built_far


#### FAR from Zoning Input

In [49]:
far_zoning = far.loc[(far.max_far_zoning == far.max_far)]
print("Number of parcels with FAR > 4 from input zoning is {}".format(len(far_zoning)))
far_zoning.head()

Number of parcels with FAR > 4 from input zoning is 3415


,parcel_id,zoning_name,max_far_zoning,juris,tra_id,pda_id,max_far_imputation,max_far_imputation_upzoning,max_far_builtfar_taz,max_far,built_far
1612240,980901,DOWNTOWN- OFFICE-500-S_pba40,33.333333,san_francisco,TRA1,San Francisco - Transbay/Rincon Hill,33.333333,33.333333,76.803463,33.333333,0.000000
964649,980892,DOWNTOWN- OFFICE-500-S_pba40,33.333333,san_francisco,TRA1,San Francisco - Transbay/Rincon Hill,33.333333,33.333333,76.803463,33.333333,0.000000
572642,980893,DOWNTOWN- OFFICE-500-S_pba40,33.333333,san_francisco,TRA1,San Francisco - Transbay/Rincon Hill,33.333333,33.333333,76.803463,33.333333,0.000000
981075,980933,PUBLIC-320-I_pba40,21.333333,san_francisco,TRA1,San Francisco - Downtown/Van Ness/Northeast Ne...,21.333333,21.333333,49.913984,21.333333,2.053832
113760,980932,DOWNTOWN- GENERAL-120/320-R-2_pba40,21.333333,san_francisco,TRA1,San Francisco - Downtown/Van Ness/Northeast Ne...,21.333333,21.333333,49.913984,21.333333,3.832590


In [50]:
far_zoning_high_nongg = far_zoning.loc[(far_zoning.max_far > 40) & 
                                       ((far_zoning.tra_id.isnull()) & (far_zoning.pda_id.isnull()))]
print("Number of parcels with FAR > 4 from input zoning and not in a TRA or PDA is {}".format(len(far_zoning_high_nongg)))
far_zoning_high_nongg.head()

Number of parcels with FAR > 4 from input zoning and not in a TRA or PDA is 0


,parcel_id,zoning_name,max_far_zoning,juris,tra_id,pda_id,max_far_imputation,max_far_imputation_upzoning,max_far_builtfar_taz,max_far,built_far


In [51]:
far_zoning_high_nongg_nonbig3 = far_zoning.loc[(far_zoning.parcel_id.isin(far_zoning_high_nongg.parcel_id)) & 
            (far_zoning.juris != 'san_francisco') & (far_zoning.juris != 'san_jose') & (far_zoning.juris != 'oakland')]
print("Number of parcels with FAR > 4 from zoning input and not in a TRA, PDA, or Big 3 is {}".format(len(far_zoning_high_nongg_nonbig3)))
far_zoning_high_nongg_nonbig3.head()

Number of parcels with FAR > 4 from zoning input and not in a TRA, PDA, or Big 3 is 0


,parcel_id,zoning_name,max_far_zoning,juris,tra_id,pda_id,max_far_imputation,max_far_imputation_upzoning,max_far_builtfar_taz,max_far,built_far
